In [ ]:
# Mail id: kedharnath1992@gmail.com
# Kindly cite if you use the script ""



#############################################
# Import necessary modules
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import math
from math import log
from sklearn.metrics import mean_squared_error

#############################################
# Read the input data and select data for analysis
df = pd.read_excel("data_constitutive_model.xlsx")
df.head()
Y02 = df.Yieldstress      # 0.2% strain
Y5 = df.Stress5strain
Y10 = df.Stress10strain
Y20 = df.Stress20strain
Y30 = df.Stress30strain
pearcol = df.drop(df.columns[[8,9,10,11]], axis ='columns')   # In python the column numbering starts from zero
X = df.drop(df.columns[[8,9,10,11]], axis ='columns')
list(X.columns)


In [ ]:
# Temperature correction to account for adiabatic heating due to work hardening

# Add isothermal data to ML (XGBoost, SHAP) after the temperature correction

from scipy.integrate import simpson
from numpy import trapz

y_all = np.dstack((Y02, Y5, Y10))   # , Y20, Y30
temp = df.Temperature
w = df.Alloying
srs = df.Strainrate

strain = np.array([0.002, 0.05, 0.1])      #  , 0.2, 0.3
delT_t = [0] * len(X)
delT_s = [0] * len(X)
temp_c = [0] * len(X)

# Density of Ta & Ta-W
for i in range(0,len(X)):
  Cp = 0.1455 + (0.09544*math.pow(10,-4)*temp[i]) + (-68.9/(temp[i]*temp[i]))
  if w[i] == 0:
    density = 16.6
  if w[i] == 2.5:
    density = 16.67     # https://www.matweb.com/search/datasheet.aspx?matguid=9bf51aa6087c4e578be0240ae02c1e17
  if w[i] == 5:
    density = 16.74
  if w[i] == 10:
    density = 16.9      # https://www.matweb.com/search/GetReference.aspx?matid=100357

  b = len(y_all[0,i])
  yline = y_all[0,i]

  x_int = [0] * b
  y_int = [0] * b
  k = 0
  for j in range(0,b):
    if (np.isnan(yline[j])) == False:    # false = value is there, so record
      x_int[k] = strain[j]
      y_int[k] = yline[j]
      k = k + 1

  integral_trap = trapz(y_int[0:k], x_int[0:k])
  integral_sim = simpson(y_int[0:k], x_int[0:k])

  delT_t[i] = (0.95*integral_trap) / (density*Cp)   # Check units: Kelvin
  delT_s[i] = (0.95*integral_sim) / (density*Cp)   # Check units: Kelvin

# This is adjusted temperature for adiabatic heating
  temp_c[i] = temp[i] + delT_t[i]

df_T = pd.DataFrame(temp_c)
df_T.to_excel('new_temp.xlsx')

In [ ]:
# Equation by Zerilli   stress = c + B exp(-bT) + K strain^n                     - Only Temp effect
# Equation by JC        stress = (A+B strain^n) (1+C ln sr) (1-T*^m)             - Temp, sr & (W) effect
# Equation by ZA        stress = co + c1 exp(-c3 T + c4 T ln sr) + c5 strain^n   - Temp, sr & (W) effect
# Equation of MTS       stress = rate-independent athermal component + rate-dependent thermal component

#########################         Input the strain and yield stress at that strain
strain = 0.1
Y10 = df.Stress10strain
#########################

Y_nan = [0] * len(X)
k = 0

for j in range(0,len(X)):
  if (np.isnan(Y10[j])) == False:
    Y_nan[k] = Y10[j]
    k = k + 1
Y = Y_nan[0:k]

stress_Z = [None] * len(Y)
stress_JC = [None] * len(Y)
stress_ZA = [None] * len(Y)
stress_MTS = [None] * len(Y)
stress_Chen = [None] * len(Y)

data = pd.DataFrame({'Strain rate (/s)': [], 'Alloying (wt% W)': [], 'Temperature (K)': [], 'Stress': [], 'Exp': []})
data_Z = pd.DataFrame({'Strain rate (/s)': [], 'Alloying (wt% W)': [], 'Temperature (K)': [], 'Stress': [], 'Exp': []})
data_JC = pd.DataFrame({'Strain rate (/s)': [], 'Alloying (wt% W)': [], 'Temperature (K)': [], 'Stress': [], 'Exp': []})
data_ZA = pd.DataFrame({'Strain rate (/s)': [], 'Alloying (wt% W)': [], 'Temperature (K)': [], 'Stress': [], 'Exp': []})
data_MTS = pd.DataFrame({'Strain rate (/s)': [], 'Alloying (wt% W)': [], 'Temperature (K)': [], 'Stress': [], 'Exp': []})

l = 0
i = 0

for i in range(0,len(X)):
  if (np.isnan(Y10[i])) == False:

    if temp[i] < 300:
      T = '77 to 300'
    elif temp[i] < 600:
      T = '300 to 600'
    elif temp[i] < 900:
      T = '600 to 900'
    else:
      T = '900 to 1273'

    if srs[i] < 0.001:
      S = '10^-6 to 10^-3'
    elif srs[i] < 1:
      S = '10^-3 to 1'
    elif srs[i] < 100:
      S = '1 to 10^2'
    else:
      S = '10^2 to 10^4'

# Equation by Zerilli   stress = c + B exp(-bT) + K strain^n           - Only Temp effect
    beta = 0.00535 - (0.000327 * np.log10(srs[i]))
    stress_Z[l] = 30 + (1125*math.exp(-beta*temp_c[i])) + (310*math.pow(strain,0.44))
    data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_Z[l]], 'Exp': [Y10[i]]})
    data_Z = pd.concat([data_Z, data], ignore_index = True)

# Equation by JC        stress = (A+B strain^n) (1+C ln sr) (1-T*^m)   - Temp, sr & (W) effect
    if w[i] == 0:
      HT = abs((temp_c[i]-296)/(3250-296))
      stress_JC[l] = (340+(750*math.pow(strain,0.7))) * (1+(0.0575*np.log10(srs[i]))) * (1-math.pow(HT,0.4))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_JC[l]], 'Exp': [Y10[i]]})
      data_JC = pd.concat([data_JC, data], ignore_index = True)
    if w[i] == 2.5:
      HT = abs((temp_c[i]-296)/(3250-296))
      stress_JC[l] = (390+(700*math.pow(strain,0.575))) * (1+(0.04*np.log10(srs[i]))) * (1-math.pow(HT,0.5))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_JC[l]], 'Exp': [Y10[i]]})
      data_JC = pd.concat([data_JC, data], ignore_index = True)
    if w[i] == 5:
      HT = abs((temp_c[i]-296)/(3250-296))
      stress_JC[l] = (400+(875*math.pow(strain,0.525))) * (1+(0.0363*np.log10(srs[i]))) * (1-math.pow(HT,0.550))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_JC[l]], 'Exp': [Y10[i]]})
      data_JC = pd.concat([data_JC, data], ignore_index = True)
    if w[i] == 10:
      HT = abs((temp_c[i]-296)/(3250-296))
      stress_JC[l] = (470+(1000*math.pow(strain,0.425))) * (1+(0.0300*np.log10(srs[i]))) * (1-math.pow(HT,0.600))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_JC[l]], 'Exp': [Y10[i]]})
      data_JC = pd.concat([data_JC, data], ignore_index = True)

# Equation by ZA        stress = co + c1 exp(-c3 T + c4 T ln sr) + c5 strain^n   - Temp, sr & (W) effect
    if w[i] == 0:
      stress_ZA[l] = 140 + (1750*math.exp((-0.00975*temp_c[i]) + (0.000675*temp_c[i]*np.log10(srs[i])))) + (650*math.pow(strain,0.650))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_ZA[l]], 'Exp': [Y10[i]]})
      data_ZA = pd.concat([data_ZA, data], ignore_index = True)
    if w[i] == 2.5:
      stress_ZA[l] = 140 + (1300*math.exp((-0.00825*temp_c[i]) + (0.000525*temp_c[i]*np.log10(srs[i])))) + (650*math.pow(strain,0.400))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_ZA[l]], 'Exp': [Y10[i]]})
      data_ZA = pd.concat([data_ZA, data], ignore_index = True)
    if w[i] == 5:
      stress_ZA[l] = 140 + (950*math.exp((-0.00650*temp_c[i]) + (0.000425*temp_c[i]*np.log10(srs[i])))) + (775*math.pow(strain,0.400))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_ZA[l]], 'Exp': [Y10[i]]})
      data_ZA = pd.concat([data_ZA, data], ignore_index = True)
    if w[i] == 10:
      stress_ZA[l] = 140 + (900*math.exp((-0.00550*temp_c[i]) + (0.000325*temp_c[i]*np.log10(srs[i])))) + (1200*math.pow(strain,0.350))
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_ZA[l]], 'Exp': [Y10[i]]})
      data_ZA = pd.concat([data_ZA, data], ignore_index = True)

# Equation of MTS       stress = rate-independent athermal component + rate-dependent thermal component
    GPa_conv = 1000
    shearmod_T = (65.25 - (0.38/(math.exp(296/temp_c[i])-1)) ) * GPa_conv # Same for all wt% of W                     # UNITS MPa
    srs_0 = 1*math.pow(10,7)
    k_b = 0.5881

    # Equation 7 from 'Deformation behavior of tantalum and a tantalum tungsten alloy', IJP, 2001
    shearmod_ratio = shearmod_T/(65.25 * GPa_conv)
    boltz = 8.63*math.pow(10,-5)                       # UNITS eV/K
    #F0 is the free energy required to overcome the barrier when the applied tau* (thermal) is zero
    F0 = 1                        # UNITS eV

    if w[i] == 0:
      # srs_0 = 5*math.pow(10,8)                       # UNITS /s    from RK, IJP, 2001
      del_g = math.pow(boltz * temp_c[i] * np.log10(srs_0/srs[i]),2/3)
      term = (1-del_g)*(1-del_g)
      stress_MTS[l] = ((130*shearmod_ratio) + (500*shearmod_ratio*math.pow(strain,0.5)) + (1140*shearmod_ratio*term)) * shearmod_ratio

      # https://doi.org/10.1007/s11665-023-08092-0
      teta = 3000
      F1 = 0.133*shearmod_T     # This is with g0i1, g0i2 is different from g0i1
      term1 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F1),2/3))),2)
      F2 = 1.6*shearmod_T
      term2 = math.pow((1-(k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F2)),3/2)
      stress_es = 650 * np.exp(k_b * temp_c[i] * np.log10(srs[i]/srs_0) / F2)
      stress_e = (np.arctanh((1-(teta/3000)) * math.tanh(2)) * stress_es / 2) / GPa_conv
      stress_Chen[l] = (0.0003*shearmod_T) + (0.0161*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      #stress_Chen[l] = (0.0003*shearmod_T) + (0.0053*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_Chen[l]], 'Exp': [Y10[i]]})
      data_MTS = pd.concat([data_MTS, data], ignore_index = True)

    if w[i] == 2.5:
      srs_0 = 3.2*math.pow(10,8)                       # UNITS /s
      del_g = math.pow(boltz * temp_c[i] * np.log10(srs_0/srs[i]),2/3)
      term = (1-del_g)*(1-del_g)
      stress_MTS[l] = ((240*shearmod_ratio) + (490*shearmod_ratio*math.pow(strain,0.4)) + (765*shearmod_ratio*term)) * shearmod_ratio

      # https://doi.org/10.1007/s11665-023-08092-0
      teta = 2000
      F1 = 0.178*shearmod_T      # This is with g0i1, g0i2 is different from g0i1
      term1 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F1),2/3))),2)
      F2 = 1.6*shearmod_T
      term2 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F2),1))),3/2)
      stress_es = 650 * np.exp(k_b * temp_c[i] * np.log10(srs[i]/srs_0) / F2)
      stress_e = (np.arctanh((1-(teta/2000)) * math.tanh(1)) * stress_es / 1) / GPa_conv
      stress_Chen[l] = (0.0003*shearmod_T) + (0.0178*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      #stress_Chen[l] = (0.0003*shearmod_T) + (0.0074*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_Chen[l]], 'Exp': [Y10[i]]})
      data_MTS = pd.concat([data_MTS, data], ignore_index = True)

    if w[i] == 5:
      srs_0 = 2*math.pow(10,8)                       # UNITS /s
      del_g = math.pow(boltz * temp_c[i] * np.log10(srs_0/srs[i]),2/3)
      term = (1-del_g)*(1-del_g)
      stress_MTS[l] = ((350*shearmod_ratio) + (480*shearmod_ratio*math.pow(strain,0.4)) + (465*shearmod_ratio*term)) * shearmod_ratio

      # https://doi.org/10.1007/s11665-023-08092-0
      teta = 2498
      F1 = 0.210*shearmod_T      # This is with g0i1, g0i2 is different from g0i1
      term1 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F1),2/3))),2)
      F2 = 1.6*shearmod_T
      term2 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F2),1))),3/2)
      stress_es = 750 * np.exp(k_b * temp_c[i] * np.log10(srs[i]/srs_0) / F2)
      stress_e = (np.arctanh((1-(teta/2500)) * math.tanh(0.5)) * stress_es / 0.5) / GPa_conv
      stress_Chen[l] = (0.0003*shearmod_T) + (0.0171*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      #tress_Chen[l] = (0.0003*shearmod_T) + (0.0084*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_Chen[l]], 'Exp': [Y10[i]]})
      data_MTS = pd.concat([data_MTS, data], ignore_index = True)

    if w[i] == 10:
      srs_0 = 1*math.pow(10,7)                       # UNITS /s     1*math.pow(10,7) from S.R. Chen, Met. Trans., 1996
      del_g = math.pow(boltz * temp_c[i] * np.log10(srs_0/srs[i]),2/3)
      term = (1-del_g)*(1-del_g)
      stress_MTS[l] = ((600*shearmod_ratio) + (450*shearmod_ratio*math.pow(strain,0.4)) + (165*shearmod_ratio*term)) * shearmod_ratio

      # https://doi.org/10.1007/s11665-023-08092-0
      teta = 3100
      F1 = 0.280*shearmod_T      # This is with g0i1, g0i2 is different from g0i1
      term1 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F1),2/3))),2)
      F2 = 1.6*shearmod_T
      term2 = math.pow((1-(math.pow((k_b*temp_c[i] * np.log10(srs_0/srs[i]) / F2),1))),3/2)
      stress_es = 850 * np.exp(k_b * temp_c[i] * np.log10(srs[i]/srs_0) / F2)
      stress_e = (np.arctanh((1-(teta/3100)) * math.tanh(0.5)) * stress_es / 0.5) / GPa_conv
      stress_Chen[l] = (0.0003*shearmod_T) + (0.0193*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      #stress_Chen[l] = (0.0003*shearmod_T) + (0.0124*shearmod_T*term1) + (stress_e*shearmod_T*term2)
      data = pd.DataFrame({'Strain rate (/s)': [S], 'Alloying (wt% W)': [w[i]], 'Temperature (K)': [T], 'Stress': [stress_Chen[l]], 'Exp': [Y10[i]]})
      data_MTS = pd.concat([data_MTS, data], ignore_index = True)

    l = l + 1

#rmse_Z = np.sqrt(mean_squared_error(stress_Z, Y))
#print(rmse_Z)
rmse_JC = np.sqrt(mean_squared_error(stress_JC, Y))
print(rmse_JC)
rmse_ZA = np.sqrt(mean_squared_error(stress_ZA, Y))
print(rmse_ZA)
#rmse_MTS = np.sqrt(mean_squared_error(stress_MTS, Y))
#print(rmse_MTS)
rmse_Chen = np.sqrt(mean_squared_error(stress_Chen, Y))
print(rmse_Chen)

# Pearson linear correlation coefficent
corr_JC, _ = pearsonr(stress_JC, Y)
print('Pearsons correlation JC: %.3f' % corr_JC)
corr_ZA, _ = pearsonr(stress_ZA, Y)
print('Pearsons correlation ZA: %.3f' % corr_ZA)
corr_MTS, _ = pearsonr(stress_MTS, Y)
print('Pearsons correlation MTS: %.3f' % corr_MTS)

In [ ]:
# Plotting

#################################  Z
plt.scatter(Y, stress_Z, c='r', label='Test data', s = 70, alpha=0.5)
plt.plot([0, 1400], [0, 1400], color = 'black', linewidth = 3, linestyle='dashed')

m = 0
for i in range(len(X)):
  if (np.isnan(Y10[i])) == False:
    plt.text(x=Y[m]+0.3,y=stress_Z[m]+0.3,s=df.index[i], fontdict=dict(color='blue',size=12))
    m = m + 1

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Experimental values', fontsize= 20, fontweight='bold')
plt.ylabel('Zerilli predicted values', fontsize= 20, fontweight='bold')
plt.legend(['Zerilli'], fontsize='x-large')
plt.savefig('Zerilli.png',dpi = 300, bbox_inches = "tight")

#################################   JC
plt.show()
plt.scatter(Y, stress_JC, c='r', label='Test data', s = 70, alpha=0.5)
plt.plot([0, 1400], [0, 1400], color = 'black', linewidth = 3, linestyle='dashed')

m = 0
for i in range(len(X)):
  if (np.isnan(Y10[i])) == False:
    plt.text(x=Y[m]+0.3,y=stress_JC[m]+0.3,s=df.index[i], fontdict=dict(color='blue',size=12))
    m = m + 1

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Experimental values', fontsize= 20, fontweight='bold')
plt.ylabel('JC predicted values', fontsize= 20, fontweight='bold')
plt.legend(['JC'], fontsize='x-large')
plt.savefig('JC.png',dpi = 300, bbox_inches = "tight")

#################################  ZA
plt.show()
plt.scatter(Y, stress_ZA, c='r', label='Test data', s = 70, alpha=0.5)
plt.plot([0, 1400], [0, 1400], color = 'black', linewidth = 3, linestyle='dashed')

m = 0
for i in range(len(X)):
  if (np.isnan(Y10[i])) == False:
    plt.text(x=Y[m]+0.3,y=stress_ZA[m]+0.3,s=df.index[i], fontdict=dict(color='blue',size=12))
    m = m + 1

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Experimental values', fontsize= 20, fontweight='bold')
plt.ylabel('ZA predicted values', fontsize= 20, fontweight='bold')
plt.legend(['ZA'], fontsize='x-large')
plt.savefig('ZA.png',dpi = 300, bbox_inches = "tight")

#################################  MTS
plt.show()
plt.scatter(Y, stress_MTS, c='r', label='Test data', s = 70, alpha=0.5)
plt.plot([0, 1400], [0, 1400], color = 'black', linewidth = 3, linestyle='dashed')

m = 0
for i in range(len(X)):
  if (np.isnan(Y10[i])) == False:
    plt.text(x=Y[m]+0.3,y=stress_MTS[m]+0.3,s=df.index[i], fontdict=dict(color='blue',size=12))
    m = m + 1

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Experimental values', fontsize= 20, fontweight='bold')
plt.ylabel('MTS predicted values', fontsize= 20, fontweight='bold')
plt.legend(['MTS'], fontsize='x-large')
plt.savefig('MTS.png',dpi = 300, bbox_inches = "tight")

#################################  Chen
plt.show()
plt.scatter(Y, stress_Chen, c='r', label='Test data', s = 70, alpha=0.5)
plt.plot([0, 1400], [0, 1400], color = 'black', linewidth = 3, linestyle='dashed')

m = 0
for i in range(len(X)):
  if (np.isnan(Y10[i])) == False:
    plt.text(x=Y[m]+0.3,y=stress_Chen[m]+0.3,s=df.index[i], fontdict=dict(color='blue',size=7))
    m = m + 1

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Experimental values', fontsize= 20, fontweight='bold')
plt.ylabel('MTS_Chen predicted values', fontsize= 20, fontweight='bold')
plt.legend(['MTS_Chen'], fontsize='x-large')
plt.savefig('Chen.png',dpi = 300, bbox_inches = "tight")

#################################  Chen
plt.show()
plt.scatter(Y, stress_Chen, c='r', label='Test data', s = 70, alpha=0.5)
plt.plot([0, 1400], [0, 1400], color = 'black', linewidth = 3, linestyle='dashed')

m = 0
for i in range(len(X)):
  if (np.isnan(Y10[i])) == False:
    plt.text(x=Y[m]+0.3,y=stress_Chen[m]+0.3,s=Y10[i], fontdict=dict(color='blue',size=7))
    m = m + 1

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Experimental values', fontsize= 20, fontweight='bold')
plt.ylabel('MTS_Chen predicted values', fontsize= 20, fontweight='bold')
plt.legend(['MTS_Chen'], fontsize='x-large')
plt.savefig('Chen.png',dpi = 300, bbox_inches = "tight")

